In [1]:
sc

In [2]:
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

In [3]:
import os

datasets_path = os.path.join('/home/hadoop', 'datasets')

complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')
small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')

In [4]:
small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')

small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]

In [5]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [6]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [7]:
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')

small_movies_raw_data = sc.textFile(small_movies_file)
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
small_movies_data.take(3)

[(u'1', u'Toy Story (1995)'),
 (u'2', u'Jumanji (1995)'),
 (u'3', u'Grumpier Old Men (1995)')]

In [8]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0L)

In [9]:
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [11]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [2,3,4,5,6,7,8]
errors = [0, 0, 0,0,0,0,0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 2 the RMSE is 0.891838015497
For rank 3 the RMSE is 0.91226920527
For rank 4 the RMSE is 0.919837162894
For rank 5 the RMSE is 0.923499064446
For rank 6 the RMSE is 0.925583764558
For rank 7 the RMSE is 0.92142359505
For rank 8 the RMSE is 0.922943984919
The best model was trained with rank 2


In [12]:
#475유저가 81132 아이템을 0.986만큼 좋아한다 예상
predictions.take(3)

[((475, 81132), 1.0111110131458738),
 ((469, 667), 3.010087433076958),
 ((181, 667), 3.026865156809132)]

In [13]:
#627유저가 30793을 3.5만큼 좋안한다했는데 3.14 로 예상함
rates_and_preds.take(3)

[((627, 30793), (3.5, 3.3268493830258086)),
 ((668, 4030), (1.5, 1.4424904872145103)),
 ((332, 2528), (4.0, 4.417437227921642))]

In [14]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

For testing data the RMSE is 0.898908775676


In [15]:
!sqoop import --connect jdbc:mysql://localhost/appingpot_db --driver com.mysql.jdbc.Driver --table  normalized_app_log -m 1 --target-dir /input/result_datas/user_Id1 --columns userId,packageId,rating --input-fields-terminated-by ',' --input-lines-terminated-by "\n"


Please set $HBASE_HOME to the root of your HBase installation.
Please set $HCAT_HOME to the root of your HCatalog installation.
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
Please set $ZOOKEEPER_HOME to the root of your Zookeeper installation.
16/10/04 18:58:32 INFO sqoop.Sqoop: Running Sqoop version: 1.4.6
16/10/04 18:58:32 WARN sqoop.ConnFactory: Parameter --driver is set to an explicit driver however appropriate connection manager is not being set (via --connection-manager). Sqoop is going to fall back to org.apache.sqoop.manager.GenericJdbcManager. Please specify explicitly which connection manager should be used next time.
16/10/04 18:58:32 INFO manager.SqlManager: Using default fetchSize of 1000
16/10/04 18:58:32 INFO tool.CodeGenTool: Beginning code generation
16/10/04 18:58:34 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM normalized_app_log AS t WHERE 1=0
16/10/04 18:58:34 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /home/hadoo

In [16]:
!hadoop fs -ls /input/result_datas/user_Id1

Found 2 items
-rw-r--r--   1 hadoop supergroup          0 2016-10-04 18:58 /input/result_datas/user_Id1/_SUCCESS
-rw-r--r--   1 hadoop supergroup        150 2016-10-04 18:58 /input/result_datas/user_Id1/part-m-00000


In [17]:
!hadoop fs -cat  /input/result_datas/user_Id1/part-m-00000

1,12,0.77506297057399
1,13,3.493399765920448
1,14,0.0
1,15,0.7862957672489755
1,16,0.42122987531195116
1,17,3.9033968445574136
1,18,8.435830302914006


In [18]:
new_user_ID = 1
new_user_ratings = sc.textFile("hdfs://175.126.112.109:9010/input/result_datas/user_Id1/part-m-00000")

In [19]:
new_user_ratings.take(5)

[u'1,12,0.77506297057399',
 u'1,13,3.493399765920448',
 u'1,14,0.0',
 u'1,15,0.7862957672489755',
 u'1,16,0.42122987531195116']

In [20]:
new_user_ratings_list=[x for x in new_user_ratings .toLocalIterator()]

In [21]:
new_user_ratings_list

[u'1,12,0.77506297057399',
 u'1,13,3.493399765920448',
 u'1,14,0.0',
 u'1,15,0.7862957672489755',
 u'1,16,0.42122987531195116',
 u'1,17,3.9033968445574136',
 u'1,18,8.435830302914006']

In [22]:
new_user_ratings_list_tuple=[]

for i in range(0,len(new_user_ratings_list)):
    userId,packageId,ratings=new_user_ratings_list[i].split(',')
    int(str(userId)),int(str(packageId)),float(str(ratings))
    #(int)userId,(int)packageId,(float)ratings
    new_user_ratings_list_tuple.append((int(str(userId)),int(str(packageId)),float(str(ratings))))
    

In [23]:
#좌변은 기존에 있던 소스에 있던 변수명 우측은 내가 파싱하면서 만들어진 변수들
new_user_ratings_RDD=sc.parallelize(new_user_ratings_list_tuple)
new_user_ratings=new_user_ratings_list_tuple
print 'New user ratings: %s' % new_user_ratings_RDD.take(10)

New user ratings: [(1, 12, 0.77506297057399), (1, 13, 3.493399765920448), (1, 14, 0.0), (1, 15, 0.7862957672489755), (1, 16, 0.42122987531195116), (1, 17, 3.9033968445574136), (1, 18, 8.435830302914006)]


In [24]:
#기존 데이터셋에 새로운 유저의 정보추가.
complete_data_with_new_ratings_RDD = training_RDD.union(new_user_ratings_RDD)

In [25]:
#새로 추가된 모델 학습시키기.
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print "New model trained in %s seconds" % round(tt,3)

New model trained in 3.604 seconds


In [26]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs of new User

In [27]:
#새로운 유저가 매긴 영화의 id
new_user_ratings_ids

[12, 13, 14, 15, 16, 17, 18]

In [28]:
#영화데이터셋 만들기
complete_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    
print "There are %s movies in the complete dataset" % (complete_movies_titles.count())

There are 10329 movies in the complete dataset


In [33]:
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
#기존 영화데이터셋-새로운 user가 등록한 movieId 만들기
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

In [34]:
new_user_unrated_movies_RDD.take(20)

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (1, 25),
 (1, 26),
 (1, 27)]

In [35]:
#Getting top recommendations
#위에서 했던거임...
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
#새로운 유저가 매기지않은 영화id(기존영화데이터셋-새로운 유저가 매긴 영화 id)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
#추천!
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

In [36]:
new_user_ratings_ids

[12, 13, 14, 15, 16, 17, 18]

In [37]:
new_user_unrated_movies_RDD.take(10)

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10)]

In [38]:
#새로 추가된 유저의 추천받은 데이터
new_user_recommendations_RDD.take(10)

[Rating(user=1, product=32196, rating=4.137270324524152),
 Rating(user=1, product=50447, rating=3.5757613584135868),
 Rating(user=1, product=68522, rating=4.195116873826848),
 Rating(user=1, product=91902, rating=3.5757613584135868),
 Rating(user=1, product=68482, rating=2.318599051953889),
 Rating(user=1, product=81132, rating=1.7878806792067934),
 Rating(user=1, product=667, rating=2.3420742613468377),
 Rating(user=1, product=44828, rating=3.0225204973933373),
 Rating(user=1, product=5618, rating=4.357870483097315),
 Rating(user=1, product=26169, rating=3.93454500970077)]

In [39]:
top_movies = new_user_recommendations_RDD.takeOrdered(25, key=lambda x: -x[2])

print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

TOP recommended movies (with more than 25 reviews):
Rating(user=1, product=80969, rating=5.959602144033809)
Rating(user=1, product=25961, rating=5.959602144033809)
Rating(user=1, product=93040, rating=5.959602144033809)
Rating(user=1, product=1085, rating=5.669256758309576)
Rating(user=1, product=6476, rating=5.573796548833734)
Rating(user=1, product=418, rating=5.527983933064579)
Rating(user=1, product=3272, rating=5.378770900916798)
Rating(user=1, product=84880, rating=5.363641928381249)
Rating(user=1, product=3311, rating=5.363641928381249)
Rating(user=1, product=1934, rating=5.363641928381249)
Rating(user=1, product=127164, rating=5.363641928381249)
Rating(user=1, product=114552, rating=5.363641928381249)
Rating(user=1, product=113064, rating=5.363641928381249)
Rating(user=1, product=31934, rating=5.363641928381249)
Rating(user=1, product=80677, rating=5.363641928381249)
Rating(user=1, product=48851, rating=5.363641928381249)
Rating(user=1, product=26003, rating=5.363641928381249)


In [40]:
top_movies

[Rating(user=1, product=80969, rating=5.959602144033809),
 Rating(user=1, product=25961, rating=5.959602144033809),
 Rating(user=1, product=93040, rating=5.959602144033809),
 Rating(user=1, product=1085, rating=5.669256758309576),
 Rating(user=1, product=6476, rating=5.573796548833734),
 Rating(user=1, product=418, rating=5.527983933064579),
 Rating(user=1, product=3272, rating=5.378770900916798),
 Rating(user=1, product=84880, rating=5.363641928381249),
 Rating(user=1, product=3311, rating=5.363641928381249),
 Rating(user=1, product=1934, rating=5.363641928381249),
 Rating(user=1, product=127164, rating=5.363641928381249),
 Rating(user=1, product=114552, rating=5.363641928381249),
 Rating(user=1, product=113064, rating=5.363641928381249),
 Rating(user=1, product=31934, rating=5.363641928381249),
 Rating(user=1, product=80677, rating=5.363641928381249),
 Rating(user=1, product=48851, rating=5.363641928381249),
 Rating(user=1, product=26003, rating=5.363641928381249),
 Rating(user=1, pr

In [41]:
#defmy_functions
user1_top_movies=sc.parallelize(top_movies)
user1_top_movies_for_hdfs=user1_top_movies.map(lambda item : str(item.user)+","+str(item.product)+","+str(item.rating))

In [42]:
user1_top_movies_for_hdfs.take(10)

['1,80969,5.95960214403',
 '1,25961,5.95960214403',
 '1,93040,5.95960214403',
 '1,1085,5.66925675831',
 '1,6476,5.57379654883',
 '1,418,5.52798393306',
 '1,3272,5.37877090092',
 '1,84880,5.36364192838',
 '1,3311,5.36364192838',
 '1,1934,5.36364192838']

In [44]:
#추천결과 HDFS저장
user1_top_movies_for_hdfs.saveAsTextFile("hdfs://175.126.112.109:9010/output/result_datas/user_Id1")

In [45]:
# 잘 저장되었나 HDFS확인
!hadoop fs -ls /output/result_datas/user1_result

Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-09-19 01:03 /output/result_datas/user1_result/_SUCCESS
-rw-r--r--   3 hadoop supergroup        546 2016-09-19 01:03 /output/result_datas/user1_result/part-00000


In [47]:
!hadoop fs -cat /output/result_datas/user1_result/part-00000

1,80969,5.85882072966
1,25961,5.85882072966
1,93040,5.85882072966
1,6476,5.4538086172
1,60503,5.30313013781
1,3272,5.27759727811
1,84880,5.27293871891
1,3311,5.27293871891
1,1934,5.27293871891
1,127164,5.27293871891
1,114552,5.27293871891
1,113064,5.27293871891
1,31934,5.27293871891
1,80677,5.27293871891
1,48851,5.27293871891
1,26003,5.27293871891
1,41627,5.27293871891
1,7088,5.27293871891
1,79830,5.27293871891
1,97880,5.27293871891
1,4801,5.27293871891
1,88646,5.27293871891
1,70849,5.27293871891
1,31309,5.27293871891
1,99912,5.27293871891


In [49]:
#sqoop을 활용하여 recommend_result 테이블에 추천결과저장
!sqoop export --connect jdbc:mysql://localhost/appingpot_db --driver com.mysql.jdbc.Driver --table recommend_result -m 1 --export-dir /output/result_datas/user1_result/part-00000 --columns userID,packageId,rating --input-fields-terminated-by ',' --input-lines-terminated-by "\n"

Please set $HBASE_HOME to the root of your HBase installation.
Please set $HCAT_HOME to the root of your HCatalog installation.
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
Please set $ZOOKEEPER_HOME to the root of your Zookeeper installation.
16/10/04 19:14:11 INFO sqoop.Sqoop: Running Sqoop version: 1.4.6
16/10/04 19:14:11 WARN sqoop.ConnFactory: Parameter --driver is set to an explicit driver however appropriate connection manager is not being set (via --connection-manager). Sqoop is going to fall back to org.apache.sqoop.manager.GenericJdbcManager. Please specify explicitly which connection manager should be used next time.
16/10/04 19:14:11 INFO manager.SqlManager: Using default fetchSize of 1000
16/10/04 19:14:11 INFO tool.CodeGenTool: Beginning code generation
16/10/04 19:14:12 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM recommend_result AS t WHERE 1=0
16/10/04 19:14:12 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /home/hadoop/